# Felipe Cardona - Tarea 3 Clasificación De Datos Utilizando Imágenes

## Configurar e Importar dependencias necesarias.

In [5]:
pip install -r requirements.txt

  Using cached tensorflow-2.17.1-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 4.2 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 22.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 20.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/382.4 MB ? eta -:--:--
   - -------------------------------------- 13.4/382.4 MB 70.1 MB/s eta 0:00:06
   -- ------------------------------------- 24.6/382.4 MB 60.0 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
import os
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

## Cargar las imagenes para entrenamiento

Al momento de cargar nos encargaremos de eliminar todas aquellas imagenes corruptas que no sean posible de leer y tambien de normalizar las imagenes

In [84]:
trainPath = 'data/train'
# Defino las dimensiones de las imágenes para estandarizar el procesamiento
IMGSIZE = 100

labelMapping = {
    "Shepherds Purse": "Shepherd’s Purse",
    # Agrega otros mapeos si es necesario
}

def loadImages(path):
    images = []
    labels = []
    for folder in os.listdir(path):
        # Obtener la etiqueta correcta usando el mapeo
        label = labelMapping.get(folder, folder)  # Usa el nombre de la carpeta o el mapeo si existe
        folderPath = os.path.join(path, folder)
        # Procesar solo si es un directorio
        if not os.path.isdir(folderPath):
            continue
        for file in glob.glob(os.path.join(folderPath, "*.png")):
            img = cv2.imread(file)
            if img is None:
                print(f"Error al cargar la imagen: {file}")
                continue
            img = cv2.resize(img, (IMGSIZE, IMGSIZE))
            img = img / 255.0  # Aplicamos normalización
            images.append(img)
            labels.append(label)  # Usar la etiqueta del mapeo en lugar del nombre de la carpeta
    return np.array(images), np.array(labels)
    
X, y = loadImages(trainPath)

### Pasaremos a codificar etiquetas y dividir los datos los datos en entrenamiento y validación

In [85]:
labelEncoder = LabelEncoder()
yEncoded = labelEncoder.fit_transform(y)
yEncoded = to_categorical(yEncoded)

# Dividir en conjuntos de entrenamiento y validación
Xtrain, Xval, yTrain, yVal = train_test_split(X, yEncoded, test_size=0.2, random_state=42)

### Pasaremos a realizar ajustes y a entrenar el modelo de Regresión Logistica

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Aplanar las imágenes
xTrainFlat = Xtrain.reshape(Xtrain.shape[0], -1)
xValFlat = Xval.reshape(Xval.shape[0], -1)

In [ ]:
logisticModel = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
logisticModel.fit(xTrainFlat, yTrain.argmax(axis=1))

C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Realizando predicciones con los datos de entrenamiento consegumos los siguientes resultados

In [68]:
yPred = logisticModel.predict(xValFlat)
accuracy = accuracy_score(yVal.argmax(axis=1), yPred)
confMatrix = confusion_matrix(yVal.argmax(axis=1), yPred)

print("Accuracy (Logistic Regression):", accuracy)
print("Confusion Matrix (Logistic Regression):\n", confMatrix)

Accuracy (Logistic Regression): 0.5074626865671642
Confusion Matrix (Logistic Regression):
 [[16  0  0  0  5 43  0  1  0  1]
 [ 0 49  5  3  1  0  0  7  5  6]
 [ 1  5 25  0  3  1  0  7  2  1]
 [ 1  1  1 58 12  0  9 22  7  6]
 [ 4  1  1  8 34 18  1  5  9  8]
 [12  0  2  3 11 88  0  1  3  6]
 [ 0  0  0 14  0  0 13 12  0  2]
 [ 3  2  4 14  6  3  4 36  1  5]
 [ 0 11  5  4  4  0  0  3 70  1]
 [ 3  0  2 10 14  7  0  9  4 19]]


### Ahora cargaremos las imagenes de test con el fin de poner a prueba la predicción del modelo.

In [58]:
testPath = "data/test"

# Función para cargar imágenes de prueba
def loadTestImages(path):
    testImages = []
    filenames = []
    filePaths = glob.glob(os.path.join(path, "*.png"))
    for file in filePaths:
        img = cv2.imread(file)
        img = cv2.resize(img, (IMGSIZE, IMGSIZE))
        img = img / 255.0
        testImages.append(img)
        filenames.append(os.path.basename(file))
    return np.array(testImages), filenames

Xtest, testFilenames = loadTestImages(testPath)

In [65]:
# Listar y ordenar alfabéticamente los archivos en el directorio de prueba
test_filenames = sorted([f for f in os.listdir(testPath) if f.endswith(".png")])

# Crear un DataFrame con los nombres de archivo ordenados alfabéticamente
test_df = pd.DataFrame({"file": test_filenames})

# Mostrar las primeras filas del DataFrame para revisión
test_df.head(10)

file_to_check = "0050ee5befb8db3ec3dc387a20f6f99a.png"
test_dir = "data/test"

# Verifica si el archivo está en el directorio
is_in_dataset = file_to_check in testFilenames

# Imprime el resultado
if is_in_dataset:
    print(f"{file_to_check} está en el dataset.")
else:
    print(f"{file_to_check} NO está en el dataset.")

0050ee5befb8db3ec3dc387a20f6f99a.png está en el dataset.


### Creamos la función que realice la predicción y genere el archivo para subir a Kaggle.

In [52]:
# Función para realizar predicciones y guardar el archivo CSV para Kaggle
def predictAndSave(model, xTest, filenames, labelEncoder, outputFile="submission.csv"):
    predictions = model.predict(xTest)
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictedLabels = labelEncoder.inverse_transform(np.argmax(predictions, axis=1))
    else:
        predictedLabels = labelEncoder.inverse_transform(predictions)
    df = pd.DataFrame({"file": filenames, "label": predictedLabels})
    df.to_csv(outputFile, index=False)

### Realizando predicción del modelo de Regresión Logistica

In [53]:
XtestFlat = Xtest.reshape(Xtest.shape[0], -1)
predictAndSave(logisticModel, XtestFlat, testFilenames, labelEncoder, outputFile="logistic_regression.csv")